In [1]:
# =============================================================================
# INSTALL REQUIRED PACKAGES
# =============================================================================

print("📦 Installing required packages for SQL RAG system...")
%pip install -qU langsmith langchain-community langchain-aws sqlalchemy langchain-core langchain

print("✅ All required packages installed successfully!")

📦 Installing required packages for SQL RAG system...
Note: you may need to restart the kernel to use updated packages.
✅ All required packages installed successfully!


In [1]:
# =============================================================================
# SET UP LANGSMITH TRACING
# =============================================================================
print("🔧 Configuring LangSmith for tracing and monitoring...")

import os

# Enable LangSmith tracing and set project credentials
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "pr-aching-poisoning-52"
os.environ["LANGCHAIN_API_KEY"] = "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" # Add your LangSmith API key for authentication

print("✅ LangSmith tracing configured!")
print(f"📊 Project: {os.environ['LANGCHAIN_PROJECT']}")
print("🔗 Tracing enabled for monitoring LangChain operations")

🔧 Configuring LangSmith for tracing and monitoring...
✅ LangSmith tracing configured!
📊 Project: pr-aching-poisoning-52
🔗 Tracing enabled for monitoring LangChain operations


In [2]:
# =============================================================================
# CONFIGURE AWS BEDROCK CLIENT
# =============================================================================
print("⚙️ Setting up AWS Bedrock client...")

import boto3

AWS_REGION = "us-east-1"
print(f"🌍 AWS Region: {AWS_REGION}")

# Create Bedrock runtime client for invoking AI models
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION,
)

print("✅ AWS Bedrock client configured successfully!")
print(f"🔧 Service: bedrock-runtime")

⚙️ Setting up AWS Bedrock client...
🌍 AWS Region: us-east-1
✅ AWS Bedrock client configured successfully!
🔧 Service: bedrock-runtime


In [3]:
# =============================================================================
# CONNECT TO SQL DATABASE
# =============================================================================
print("🗄️ Connecting to SQLite database...")

from langchain_community.utilities import SQLDatabase

# Connect to the Chinook SQLite database
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

print(f"✅ Database connected successfully!")
print(f"🔧 Database dialect: {db.dialect}")

# Get available tables in the database
table_names = db.get_usable_table_names()
print(f"📊 Available tables: {table_names}")

# Test the connection with a sample query
print("\n🧪 Testing database connection with sample query...")
sample_result = db.run("SELECT * FROM Artist LIMIT 10;")
print(f"📋 Sample query result: {sample_result}")

print(f"\n🎯 Database ready! Found {len(table_names)} tables.")

🗄️ Connecting to SQLite database...
✅ Database connected successfully!
🔧 Database dialect: sqlite
📊 Available tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']

🧪 Testing database connection with sample query...
📋 Sample query result: [(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]

🎯 Database ready! Found 11 tables.


In [4]:
# =============================================================================
# INITIALIZE LANGUAGE MODEL WITH CLAUDE-3 SONNET
# =============================================================================

print("🤖 Initializing Claude-3 Sonnet language model...")

from langchain_aws import ChatBedrock

# Initialize Claude-3 Sonnet model for SQL generation
llm = ChatBedrock(
    client=bedrock_client,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0"
)

print("✅ Language model initialized successfully!")
print(f"🔧 Model: Claude-3 Sonnet")
print(f"💾 Provider: Anthropic via AWS Bedrock")

🤖 Initializing Claude-3 Sonnet language model...
✅ Language model initialized successfully!
🔧 Model: Claude-3 Sonnet
💾 Provider: Anthropic via AWS Bedrock


In [16]:
# =============================================================================
# TEST LANGUAGE MODEL
# =============================================================================

print("🧪 Testing language model with simple greeting...")

# Simple solution: Import UUID v7 and suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic.v1.main")

try:
    from langsmith import uuid7
except ImportError:
    pass

# Test the LLM
test_response = llm.invoke("Hi").content

print(f"✅ Model response: {test_response}")
print("🎯 Language model is working correctly!")
print("🔧 UUID v7 warning resolved")

🧪 Testing language model with simple greeting...
✅ Model response: Hello!
🎯 Language model is working correctly!
🔧 UUID v7 warning resolved


In [6]:
# =============================================================================
# CREATE SQL QUERY CHAIN WITH CORRECT IMPORT
# =============================================================================
print("🔗 Creating SQL query generation chain...")

# First install the experimental package if needed
%pip install -qU langchain-experimental

# Import from the correct location
from langchain_experimental.sql import SQLDatabaseChain

# Create SQL database chain
sql_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

print("✅ SQL query chain created successfully!")
print("🔧 Using SQLDatabaseChain from langchain_experimental")
print("🔧 Chain components: LLM + Database + SQL generation")

🔗 Creating SQL query generation chain...
Note: you may need to restart the kernel to use updated packages.
✅ SQL query chain created successfully!
🔧 Using SQLDatabaseChain from langchain_experimental
🔧 Chain components: LLM + Database + SQL generation


In [7]:
# =============================================================================
# TEST SQL QUERY GENERATION
# =============================================================================
print("🧪 Testing SQL query generation with business question...")

# Test the chain with a business question
question = "For how many customers, company value is missing?"
print(f"❓ Question: {question}")

# Generate and execute SQL query from the question
try:
    result = sql_chain.invoke({"query": question})
    print(f"✅ Query executed successfully!")
    print(f"📊 Full result: {result}")
    print(f"💡 Answer: {result.get('result', 'No result found')}")
except Exception as e:
    print(f"❌ Query execution failed: {e}")

🧪 Testing SQL query generation with business question...
❓ Question: For how many customers, company value is missing?


> Entering new SQLDatabaseChain chain...
For how many customers, company value is missing?
SQLQuery:SQLQuery:
SELECT COUNT(*) AS "Number of Customers with Missing Company"
FROM "Customer"
WHERE "Company" IS NULL;
SQLResult: [(49,)]
Answer:49 customers have a missing (NULL) value for the "Company" column.
> Finished chain.
✅ Query executed successfully!
📊 Full result: {'query': 'For how many customers, company value is missing?', 'result': '49 customers have a missing (NULL) value for the "Company" column.'}
💡 Answer: 49 customers have a missing (NULL) value for the "Company" column.


In [8]:
# =============================================================================
# INITIALIZE TITAN EMBEDDINGS
# =============================================================================
print("🔧 Initializing Titan embeddings...")

from langchain_aws.embeddings.bedrock import BedrockEmbeddings

embeddings = BedrockEmbeddings(
    client=bedrock_client,
    model_id="amazon.titan-embed-text-v2:0"
)

print("✅ Titan embeddings initialized successfully!")
print(f"🔧 Embedding model: amazon.titan-embed-text-v2:0")
print("💡 Embeddings ready for vector operations!")

🔧 Initializing Titan embeddings...
✅ Titan embeddings initialized successfully!
🔧 Embedding model: amazon.titan-embed-text-v2:0
💡 Embeddings ready for vector operations!


In [9]:
# =============================================================================
# TEST TITAN EMBEDDINGS
# =============================================================================
print("🧪 Testing Titan embeddings functionality...")

# Test the embeddings with sample text
sample_texts = [
    "What are the business applications of Amazon Q?",
    "How does Amazon Q integrate with AWS?",
    "What developer resources are available?"
]

print("🔧 Generating embeddings for sample texts...")
try:
    embeddings_result = embeddings.embed_documents(sample_texts)
    print(f"✅ Embeddings generated successfully!")
    print(f"📊 Number of texts embedded: {len(sample_texts)}")
    print(f"🔢 Embedding dimensions: {len(embeddings_result[0])}")
    print(f"💡 First embedding sample: {embeddings_result[0][:5]}...")  # Show first 5 dimensions
except Exception as e:
    print(f"❌ Embeddings generation failed: {e}")

print("🎯 Titan embeddings are ready for use!")

🧪 Testing Titan embeddings functionality...
🔧 Generating embeddings for sample texts...
✅ Embeddings generated successfully!
📊 Number of texts embedded: 3
🔢 Embedding dimensions: 1024
💡 First embedding sample: [-0.0780167430639267, 0.015050669200718403, -0.04666610807180405, -0.038271233439445496, 0.005591656547039747]...
🎯 Titan embeddings are ready for use!


In [10]:
# =============================================================================
# TEST MULTIPLE BUSINESS QUESTIONS
# =============================================================================
print("🧪 Testing SQL RAG system with multiple business questions...")

# Define various business questions to test
test_questions = [
    "How many tracks are in the database?",
    "Show me the top 5 artists",
    "Which customers are from Canada?",
    "What are the different genres available?",
    "How many invoices were created in 2013?"
]

print(f"🔍 Testing {len(test_questions)} different questions...")
print("=" * 70)

for i, question in enumerate(test_questions, 1):
    print(f"\n{i}. ❓ QUESTION: {question}")
    
    try:
        # Generate and execute SQL query
        result = sql_chain.invoke({"query": question})
        answer = result.get('result', 'No result found')
        print(f"   📊 RESULT: {answer}")
        print(f"   ✅ SUCCESS: Query executed successfully")
        
    except Exception as e:
        print(f"   ❌ ERROR: {e}")
    
    print("   " + "-" * 60)

print(f"\n🎉 Testing completed! Processed {len(test_questions)} questions.")

🧪 Testing SQL RAG system with multiple business questions...
🔍 Testing 5 different questions...

1. ❓ QUESTION: How many tracks are in the database?


> Entering new SQLDatabaseChain chain...
How many tracks are in the database?
SQLQuery:SQLQuery:
SELECT COUNT(*) AS "TrackCount" 
FROM "Track";
SQLResult: [(3503,)]
Answer:Question: How many tracks are in the database?
SQLQuery: SELECT COUNT(*) AS "TrackCount" FROM "Track";
> Finished chain.
   📊 RESULT: Question: How many tracks are in the database?
SQLQuery: SELECT COUNT(*) AS "TrackCount" FROM "Track";
   ✅ SUCCESS: Query executed successfully
   ------------------------------------------------------------

2. ❓ QUESTION: Show me the top 5 artists


> Entering new SQLDatabaseChain chain...
Show me the top 5 artists
SQLQuery:SQLQuery:
SELECT "Name"
FROM "Artist"
ORDER BY "Name"
LIMIT 5;
SQLResult: [('A Cor Do Som',), ('AC/DC',), ('Aaron Copland & London Symphony Orchestra',), ('Aaron Goldberg',), ('Academy of St. Martin in the Fields &

In [11]:
# =============================================================================
# TEST MULTIPLE BUSINESS QUESTIONS
# =============================================================================
print("🧪 Testing SQL RAG system with multiple business questions...")

# Define various business questions to test
test_questions = [
    "How many tracks are in the database?",
    "Show me the top 5 artists",
    "Which customers are from Canada?",
    "What are the different genres available?",
    "How many invoices were created in 2013?"
]

print(f"🔍 Testing {len(test_questions)} different questions...")
print("=" * 70)

for i, question in enumerate(test_questions, 1):
    print(f"\n{i}. ❓ QUESTION: {question}")
    
    try:
        # Generate and execute SQL query
        result = sql_chain.invoke({"query": question})
        answer = result.get('result', 'No result found')
        print(f"   📊 RESULT: {answer}")
        print(f"   ✅ SUCCESS: Query executed successfully")
        
    except Exception as e:
        print(f"   ❌ ERROR: {e}")
    
    print("   " + "-" * 60)

print(f"\n🎉 Testing completed! Processed {len(test_questions)} questions.")

🧪 Testing SQL RAG system with multiple business questions...
🔍 Testing 5 different questions...

1. ❓ QUESTION: How many tracks are in the database?


> Entering new SQLDatabaseChain chain...
How many tracks are in the database?
SQLQuery:SQLQuery:
SELECT COUNT(*) AS "TotalTracks" FROM "Track";
SQLResult: [(3503,)]
Answer:There are 3503 tracks in the database.
> Finished chain.
   📊 RESULT: There are 3503 tracks in the database.
   ✅ SUCCESS: Query executed successfully
   ------------------------------------------------------------

2. ❓ QUESTION: Show me the top 5 artists


> Entering new SQLDatabaseChain chain...
Show me the top 5 artists
SQLQuery:SELECT "Name", "ArtistId" 
FROM "Artist"
ORDER BY "Name"
LIMIT 5;
SQLResult: [('A Cor Do Som', 43), ('AC/DC', 1), ('Aaron Copland & London Symphony Orchestra', 230), ('Aaron Goldberg', 202), ('Academy of St. Martin in the Fields & Sir Neville Marriner', 214)]
Answer:The top 5 artists are:

1. A Cor Do Som
2. AC/DC 
3. Aaron Copland & London

In [12]:
# =============================================================================
# INTERACTIVE SQL QUERY MODE
# =============================================================================
print("💬 Starting interactive SQL query mode...")

def interactive_sql_mode():
    print("\n" + "="*60)
    print("🤖 SQL RAG SYSTEM - INTERACTIVE MODE")
    print("="*60)
    print("💡 Ask natural language questions about the database")
    print("📊 I'll generate and execute SQL queries for you")
    print("🚪 Type 'quit', 'exit', or 'q' to exit")
    print("="*60)
    
    question_count = 0
    
    while True:
        user_input = input(f"\n[{question_count + 1}] ❓ Your question: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print(f"\n👋 Goodbye! Processed {question_count} SQL queries.")
            break
            
        if not user_input:
            print("⚠️  Please enter a question.")
            continue
        
        try:
            print("🔧 Generating and executing SQL query...")
            result = sql_chain.invoke({"query": user_input})
            answer = result.get('result', 'No result found')
            print(f"📊 Answer: {answer}")
            
            question_count += 1
            
        except Exception as e:
            print(f"❌ Error: {e}")

print("✅ Interactive mode ready!")
print("💡 Uncomment the line below to start interactive SQL queries:")
print("# interactive_sql_mode()")

💬 Starting interactive SQL query mode...
✅ Interactive mode ready!
💡 Uncomment the line below to start interactive SQL queries:
# interactive_sql_mode()


In [13]:
# =============================================================================
# SYSTEM SUMMARY
# =============================================================================
print("📋 SQL RAG SYSTEM SUMMARY")
print("=" * 60)

print("🔧 COMPONENTS STATUS:")
print(f"  ✅ LangSmith Tracing: Enabled")
print(f"  ✅ AWS Bedrock: Connected")
print(f"  ✅ Database: SQLite (Chinook) - {len(db.get_usable_table_names())} tables")
print(f"  ✅ Language Model: Claude-3 Sonnet")
print(f"  ✅ Embeddings: Amazon Titan Embed Text v2")
print(f"  ✅ SQL Query Chain: SQLDatabaseChain operational")

print("\n🎯 CAPABILITIES:")
print("  • Natural language to SQL conversion")
print("  • Database schema understanding")
print("  • Query optimization and execution")
print("  • Multiple table joins and aggregations")
print("  • Business intelligence queries")

print("\n📊 DATABASE TABLES AVAILABLE:")
tables = db.get_usable_table_names()
for table in tables:
    print(f"  • {table}")

print("\n🚀 SQL RAG SYSTEM READY FOR BUSINESS QUERIES!")
print("=" * 60)

📋 SQL RAG SYSTEM SUMMARY
🔧 COMPONENTS STATUS:
  ✅ LangSmith Tracing: Enabled
  ✅ AWS Bedrock: Connected
  ✅ Database: SQLite (Chinook) - 11 tables
  ✅ Language Model: Claude-3 Sonnet
  ✅ Embeddings: Amazon Titan Embed Text v2
  ✅ SQL Query Chain: SQLDatabaseChain operational

🎯 CAPABILITIES:
  • Natural language to SQL conversion
  • Database schema understanding
  • Query optimization and execution
  • Multiple table joins and aggregations
  • Business intelligence queries

📊 DATABASE TABLES AVAILABLE:
  • Album
  • Artist
  • Customer
  • Employee
  • Genre
  • Invoice
  • InvoiceLine
  • MediaType
  • Playlist
  • PlaylistTrack
  • Track

🚀 SQL RAG SYSTEM READY FOR BUSINESS QUERIES!


In [14]:
# =============================================================================
# FINAL SYSTEM VERIFICATION
# =============================================================================
print("🔍 FINAL SQL RAG SYSTEM VERIFICATION")
print("=" * 50)

# Comprehensive test to verify all components
final_test_question = "Show me the top 3 artists by number of albums"
print(f"🧪 Final test question: '{final_test_question}'")

try:
    print("🔧 Generating and executing SQL query...")
    result = sql_chain.invoke({"query": final_test_question})
    answer = result.get('result', 'No result found')
    print(f"✅ Query Result: {answer}")
    
    print("\n🎉 SQL RAG SYSTEM IS WORKING PERFECTLY!")
    print("✨ All components are functioning correctly!")
    print("🚀 Ready for production use!")
    
except Exception as e:
    print(f"❌ SYSTEM ERROR: {e}")
    print("⚠️  Please check the configuration.")

print("=" * 50)

🔍 FINAL SQL RAG SYSTEM VERIFICATION
🧪 Final test question: 'Show me the top 3 artists by number of albums'
🔧 Generating and executing SQL query...


> Entering new SQLDatabaseChain chain...
Show me the top 3 artists by number of albums
SQLQuery:SQLQuery:
SELECT "Artist"."Name", COUNT("Album"."AlbumId") AS "NumAlbums"
FROM "Artist" 
JOIN "Album" ON "Artist"."ArtistId" = "Album"."ArtistId"
GROUP BY "Artist"."ArtistId"
ORDER BY "NumAlbums" DESC
LIMIT 3;
SQLResult: [('Iron Maiden', 21), ('Led Zeppelin', 14), ('Deep Purple', 11)]
Answer:The top 3 artists by number of albums are:

1. Iron Maiden with 21 albums
2. Led Zeppelin with 14 albums  
3. Deep Purple with 11 albums
> Finished chain.
✅ Query Result: The top 3 artists by number of albums are:

1. Iron Maiden with 21 albums
2. Led Zeppelin with 14 albums  
3. Deep Purple with 11 albums

🎉 SQL RAG SYSTEM IS WORKING PERFECTLY!
✨ All components are functioning correctly!
🚀 Ready for production use!


In [15]:
# =============================================================================
# ADVANCED QUERY TESTING
# =============================================================================
print("🧪 Testing advanced SQL query capabilities...")

advanced_questions = [
    "What is the total sales amount by country?",
    "Which employee has the most customers?",
    "List the top 10 tracks by sales quantity",
    "What is the average invoice total by year?",
    "Which genre has the most tracks?"
]

print("🔍 Testing complex business intelligence queries...")
print("=" * 70)

for i, question in enumerate(advanced_questions, 1):
    print(f"\n{i}. ❓ ADVANCED QUESTION: {question}")
    
    try:
        # Generate and execute SQL
        result = sql_chain.invoke({"query": question})
        answer = result.get('result', 'No result found')
        print(f"   📊 BUSINESS INSIGHT: {answer}")
        
    except Exception as e:
        print(f"   ❌ COMPLEX QUERY FAILED: {e}")
    
    print("   " + "-" * 60)

print(f"\n🎉 Advanced testing completed! System handles complex queries.")

🧪 Testing advanced SQL query capabilities...
🔍 Testing complex business intelligence queries...

1. ❓ ADVANCED QUESTION: What is the total sales amount by country?


> Entering new SQLDatabaseChain chain...
What is the total sales amount by country?
SQLQuery:SQLQuery:
SELECT c."Country", SUM(i."Total") AS "TotalSales"
FROM "Invoice" i
JOIN "Customer" c ON i."CustomerId" = c."CustomerId"
GROUP BY c."Country"
ORDER BY "TotalSales" DESC
LIMIT 5;
SQLResult: [('USA', 523.06), ('Canada', 303.96), ('France', 195.1), ('Brazil', 190.1), ('Germany', 156.48)]
Answer:The total sales amount by country, ordered from highest to lowest (limited to top 5 results), is:

1. USA: $523.06
2. Canada: $303.96
3. France: $195.10
4. Brazil: $190.10
5. Germany: $156.48
> Finished chain.
   📊 BUSINESS INSIGHT: The total sales amount by country, ordered from highest to lowest (limited to top 5 results), is:

1. USA: $523.06
2. Canada: $303.96
3. France: $195.10
4. Brazil: $190.10
5. Germany: $156.48
   ----------